In [1]:
# from __future__ import print_function
# from __future__ import division

# Torch
import torch
import torch.utils.data as data
from torchsummary import summary

# Numpy, scipy, scikit-image, spectral
import numpy as np
import sklearn.svm
import sklearn.model_selection
from skimage import io

# Visualization
import seaborn as sns
# import visdom

import os
from utils import (
    convert_to_color_,
    convert_from_color_,
    get_device,
    # metrics,
    # display_dataset,
    # display_predictions,
    # explore_spectrums,
    # plot_spectrums,
    # sample_gt,
    # build_dataset,
    # show_results,
    # compute_imf_weights,
)
# from datasets import get_dataset, HyperX, open_file, DATASETS_CONFIG
from datasets import load_dataset
from models import get_model #, train, test, save_model

In [2]:
CUDA_DEVICE = get_device(0)

# % of training samples
SAMPLE_PERCENTAGE = 10
# Data augmentation ?
# FLIP_AUGMENTATION = args.flip_augmentation
# RADIATION_AUGMENTATION = args.radiation_augmentation
# MIXTURE_AUGMENTATION = args.mixture_augmentation
# Dataset name
DATASET = 'Salinas'
# Model name
MODEL = 'hamida'
# Number of runs (for cross-validation)
N_RUNS = 1
# Spatial context size (number of neighbours in each spatial direction)
PATCH_SIZE = 5
# Add some visualization of the spectra ?
# DATAVIZ = args.with_exploration

# Target folder to store/download/load the datasets
FOLDER = 'Datasets\Raw\Salinas'
# Number of epochs to run
EPOCH = 5
# Sampling mode, e.g random sampling
SAMPLING_MODE = 'random'
# Pre-computed weights to restore
CHECKPOINT = None
# Learning rate for the SGD
LEARNING_RATE = 0.01
# Automated class balancing
CLASS_BALANCING = False
# Training ground truth file
# TRAIN_GT = args.train_set
# Testing ground truth file
# TEST_GT = args.test_set
TEST_STRIDE = 1


/!\ CUDA was requested but is not available! Computation will go on CPU. /!\


In [3]:
# Load the dataset
df, img, gt, LABEL_VALUES, IGNORED_LABELS, RGB_BANDS, palette = load_dataset(dataset_name='Salinas',
                                                                         dr_method='PCA',
                                                                         n_components=8)
# Number of classes
N_CLASSES = len(LABEL_VALUES)
# Number of bands (last dimension of the image tensor)
N_BANDS = img.shape[-1]

if palette is None:
    # Generate color palette
    palette = {0: (0, 0, 0)}
    for k, color in enumerate(sns.color_palette("hls", len(LABEL_VALUES) - 1)):
        palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype="uint8"))
invert_palette = {v: k for k, v in palette.items()}


def convert_to_color(x):
    return convert_to_color_(x, palette=palette)


def convert_from_color(x):
    return convert_from_color_(x, palette=invert_palette)

color_gt = convert_to_color(gt)

In [31]:
from sklearn.model_selection import train_test_split

results = []
# run the experiment several times
for run in range(N_RUNS):
    X_train, X_test, train_gt, test_gt = train_test_split(img, gt, test_size=1-SAMPLE_PERCENTAGE/100)
    print(
        "{} samples selected (over {})".format(
            np.count_nonzero(train_gt), np.count_nonzero(gt)
        )
    )
    print(
        "Running an experiment with the {} model".format(MODEL),
        "run {}/{}".format(run + 1, N_RUNS),
    )
    
    if MODEL == 'nearest':
        #...
        print()
    else:
        # if CLASS_BALANCING:
        #     weights = compute_imf_weights(train_gt, N_CLASSES, IGNORED_LABELS)
        #     hyperparams["weights"] = torch.from_numpy(weights)
        # Neural network
        # model, optimizer, loss, hyperparams = get_model(MODEL, **hyperparams)

        BATCH_SIZE=100

        model, optimizer, loss = get_model(MODEL, device=CUDA_DEVICE, n_classes=N_CLASSES, n_bands=N_BANDS, 
                                           ignored_labels=IGNORED_LABELS, patch_size=PATCH_SIZE, dropout=False, 
                                           lr=LEARNING_RATE, epoch=EPOCH, batch_size=BATCH_SIZE)
        
        train_loader = data.DataLoader(df, 
                                        batch_size=BATCH_SIZE,
                                        shuffle=True)
        
        print("Network :")
        with torch.no_grad():
            for input in df.iterrows():
                break
            print(input)
            summary(model.to(CUDA_DEVICE), input[1])
            # We would like to use device=hyperparams['device'] altough we have
            # to wait for torchsummary to be fixed first.
        # Split train set in train/val


5394 samples selected (over 54129)
Running an experiment with the hamida model run 1/1
Network :
(0, PC-1    -6136.415770
PC-2    -4385.651601
PC-3      628.426690
PC-4     -302.089387
PC-5       -7.164320
PC-6       48.222012
PC-7     -129.801973
PC-8      -28.045864
class       0.000000
Name: 0, dtype: float64)


c:\Users\Dell\anaconda3\envs\dslab1\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


TypeError: Value after * must be an iterable, not float

In [29]:
input[1].shape[-1]

9